# Detecção de objetos com TPUs
Siga os passos descritos neste notebook para executar seu modelo de detecção de objetos utilizando TPUs. Os detalhes completos de como isso é possível podem ser vistos aqui: 

**Nota:** Antes de mais nada, tenha certeza que seu ambiente de execução está configurado para usar TPUs. Para isso, acesse "Ambiente de execução" - "Alterar ambiente de execução".

## Importando os pacotes necessários e iniciando sessão
Primeiro é preciso importar as bibliotecas que serão utilizadas no notebook. Feito isso, é preciso se autenticar em sua conta do Google, garantindo acesso ao bucket em que os arquivos TFRecord foram armazenados.

In [1]:
import os
from google.colab import auth
import tensorflow as tf

In [2]:
auth.authenticate_user()

# Conectando-se ao cluster de TPU
Ao executar o código abaixo, você deverá se conectar a um cluster de TPUs que será utilizado para executar o treinamento do modelo!

In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.108.35.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.108.35.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Configurando os hiperparâmetros
Feito isso, você deverá configurar os hiperparâmetros que serão utilizados para o treinamento do modelo, como número de steps e batch size. O arquivo de configuração da pipeline pode ser carregado diretamente no google colab (conforme o código em exemplo) ou colocado no bucket do Google Cloud Platform.

In [8]:
batch_size = 8 * strategy.num_replicas_in_sync # O batch se multiplica por cada nó da TPU
num_eval_steps = 50
num_steps = 1000
pipeline_fname = '/content/pipeline_tpu.config'
model_dir = 'gs://your-cloud-bucket/model/'

## Instalação das dependências
O código abaixo faz a instalação das dependências necessárias para execução do treinamento da detecção de objetos.

In [5]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py . 
!python -m pip install .

/content
/content/models/research
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.4 MB/s 
     |████████████████████████████████| 352 kB 68.5 MB/s 
     |████████████████████████████████| 1.8 MB 12.5 MB/s 
     |████████████████████████████████| 636 kB 44.4 MB/s 
     |████████████████████████████████| 90 kB 5.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 99 kB 7.0 MB/s 
     |████████████████████████████████| 4.4 MB 25.2 MB/s 
     |████████████████████████████████| 37.1 MB 117 kB/s 
     |███████████

## Executando o treinamento
Por fim, basta executar o comando abaixo e o treinamento do seu modelo será iniciado.

In [9]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --use_tpu=True\
    --checkpoint_every_n=1000 \
    --num_eval_steps={num_eval_steps}

2021-10-08 00:13:27.247706: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-08 00:13:27.247765: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0e2849afcc72): /proc/driver/nvidia/version does not exist
2021-10-08 00:13:27.253846: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.108.35.130:8470}
2021-10-08 00:13:27.253915: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:33344}
2021-10-08 00:13:27.271684: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.108.35.130:8470}
2021-10-08 00:13:27.271790: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> 